In [1]:
import pandas

In [6]:
%%time
df = pandas.read_excel( '../assets/DOHMH_New_York_City_Restaurant_Inspection_Results.xlsx' )

CPU times: user 1min 20s, sys: 503 ms, total: 1min 20s
Wall time: 1min 20s


In [7]:
df.columns

Index(['CAMIS', 'DBA', 'BORO', 'BUILDING', 'STREET', 'ZIPCODE', 'PHONE',
       'CUISINE DESCRIPTION', 'INSPECTION DATE', 'ACTION', 'VIOLATION CODE',
       'VIOLATION DESCRIPTION', 'CRITICAL FLAG', 'SCORE', 'GRADE',
       'GRADE DATE', 'RECORD DATE', 'INSPECTION TYPE'],
      dtype='object')

In [8]:
df.head(4)

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE
0,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462,7188924968,Bakery,2015-02-09,Violations were cited in the following area(s).,06C,Food not protected from potential source of co...,Critical,6,A,2015-02-09,2015-08-14,Cycle Inspection / Initial Inspection
1,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462,7188924968,Bakery,2014-03-03,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,2,A,2014-03-03,2015-08-14,Cycle Inspection / Initial Inspection
2,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462,7188924968,Bakery,2013-10-10,No violations were recorded at the time of thi...,NaN,NaN,Not Applicable,NaN,NaN,NaT,2015-08-14,Trans Fat / Second Compliance Inspection
3,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462,7188924968,Bakery,2013-09-11,Violations were cited in the following area(s).,04L,Evidence of mice or live mice present in facil...,Critical,6,A,2013-09-11,2015-08-14,Cycle Inspection / Re-inspection


In [9]:
df.shape

(492975, 18)

In [10]:
df2 = pandas.read_csv( '../_data/DOHNYC.csv')